# Demand prediction baseline solution

Отчет.
1. Постановка задачи: по данным о продажах за 2012-2014 годы предсказать продажи конкретных товаров в 2015 году.
2. Итоговое решение: в ходе изучения данных было установлено, что столбцы 'f[i]' и 'f[i+30]' идентичны для каждой строки исходных данных. В связи с этим выбрасываем половину из них.
3. Подходы: 

    1) Использование метрики качества MAE. В связи с тем, что оценка качества производится с помощью $SMAPE = \frac{200%}{n}\sum_{n=1}^{N}\frac{|y - predict|}{|y| + |predict|}$, было принято решение минимизировать MAE. Это дало некоторое преимущество и улучшило результат.
    
    2) Выбор недель. Тестовые данные относятся только к 3-5 неделям 2015 года. Посколько речь идет о продажах, которые могут зависеть от сезона, возникло предположение, что стоит обучать модель только на той части тренировочной выборки, которая относится к 3-5 неделям. Улучшило результат.
    
    3) Кодирование item_id. Попытка заменить item_id на средние значения тех или иных фич только ухудшила прогноз. Нормировка и взаимодействие со значениями item_id внутри столбца не принесло никаких результатов.
    
    4) Изменение параметров модели. Увеличивая n_estimators и их максимальную глубину, получили несколько переобученную модель, дающую при этом хорошие результаты. Хорошо увеличило качество.
    
    5) Работа со сдвигом. Попытка обучения различных моделей для каждого из сдвигов дала результат несколько хуже, чем приведенная ниже модель.
4. Код ниже
5. Приоценке качества использовалась функция SMAPE

In [29]:
import pandas as pd
import numpy as np

train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")
sample_submission = pd.read_csv("sample_submission.tsv")

In [30]:
frac = 0.5 # fraction of learning examples used for model fitting
for i in range(31, 61):
    st = 'f' + str(i)
    train = train.drop([st], axis = 1)
    test = test.drop([st], axis = 1)
    
#sample the train set if your don't want to deel with all examples
X = train.sample(frac=frac, random_state=42)
X = X[X['week'].isin([3, 4, 5])]
y = X['y'].copy()
X = X.drop(['Num','y'], axis=1)

In [42]:
def SMAPE(pred, y):
        return float(200) / len(y) * sum(abs(pred - y) / (abs(y) + abs(pred)))

In [43]:
from sklearn import cross_validation

trainX, testX, trainY, testY = cross_validation.train_test_split(X, y, test_size=0.20, random_state=1)

In [44]:
import xgboost as xgb

model = xgb.XGBRegressor(n_estimators=800, max_depth=20, learning_rate=0.05)
model.fit(trainX, trainY, eval_metric="mae")

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.05, max_delta_step=0, max_depth=20,
       min_child_weight=1, missing=None, n_estimators=800, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [45]:
preds = model.predict(testX)

In [46]:
SMAPE(preds, np.array(testY))

22.651751127724911

In [36]:
import xgboost as xgb

model = xgb.XGBRegressor(n_estimators=800, max_depth=20, learning_rate=0.05)
model.fit(X, y, eval_metric="mae")
preds = model.predict(test.drop(['Num'], axis=1))

In [37]:
sample_submission['y'] = preds

In [38]:
sample_submission.head(5)

,Num,y
0,348622,1771.993164
1,348623,24432.169922
2,348624,338828.687500
3,348625,31894.257812
4,348626,292.033722


In [39]:
# In GBM you can get some negative predictions:
print sample_submission[sample_submission['y'] < 0]

Empty DataFrame
Columns: [Num, y]
Index: []


In [40]:
sample_submission['y'] = sample_submission['y'].map(lambda x: x if x > 0 else 0.0)

In [41]:
sample_submission.to_csv("baseline_submission.tsv", sep=',', index=False)